In [1]:
'''
'''

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.utils.training_utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint, TensorBoard

from collections import Counter

import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


In [2]:
# path to the model weights files.
# weights_path = '../keras/examples/vgg16_weights.h5'
# top_model_weights_path = 'fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '/subset_data/training_data/train'
validation_data_dir = '/subset_data/training_data/val'
checkpoint_dir = '/subset_data/ResNet50/MobileNetV2_20190323_weights.h5'
tensorboard_dir = '/subset_data/ResNet50/'
nb_train_samples = 25000
nb_validation_samples = 5000
epochs = 25
batch_size = 32
# gpu_count = 2

In [3]:
# build the MobileNetV2 network
ResNet50 = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
model = Sequential()
model.add(ResNet50)
model.add(Flatten(input_shape=model.output_shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(15, activation='softmax'))

# model = multi_gpu_model(model, gpus=gpu_count)

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
# top_model.load_weights(top_model_weights_path)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
# for layer in model.layers[:25]:
#     layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=checkpoint_dir, verbose=1, save_best_only=True)
tboard = TensorBoard(tensorboard_dir)

Instructions for updating:
Colocations handled automatically by placer.


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model loaded.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)        

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=True)

counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}  

Found 307929 images belonging to 15 classes.
Found 65983 images belonging to 15 classes.


In [5]:
class_weights

{0: 1.1302171631549205,
 1: 3.930155521868508,
 2: 6.776379477250726,
 3: 4.5564017444509535,
 4: 1.0,
 5: 132.82732447817835,
 6: 4.5158376878911035,
 7: 1.0,
 8: 9.775171065493646,
 9: 10.03584229390681,
 10: 57.995028997514495,
 11: 3.843830651803855,
 12: 7.353713625380817,
 13: 22.89826627412496,
 14: 210.84337349397592}

In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size, 
    class_weight=class_weights, 
    max_queue_size=batch_size*4,
    callbacks=[checkpointer, tboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
781/781 [==============================] - 1201s 2s/step - loss: 11.0462 - acc: 0.2292 - val_loss: 1.9324 - val_acc: 0.3401

Epoch 00001: val_loss improved from inf to 1.93243, saving model to /subset_data/ResNet50/MobileNetV2_20190323_weights.h5
Epoch 2/25
781/781 [==============================] - 1173s 2s/step - loss: 7.0728 - acc: 0.3572 - val_loss: 1.8782 - val_acc: 0.3634

Epoch 00002: val_loss improved from 1.93243 to 1.87818, saving model to /subset_data/ResNet50/MobileNetV2_20190323_weights.h5
Epoch 3/25
781/781 [==============================] - 1173s 2s/step - loss: 5.7226 - acc: 0.4545 - val_loss: 2.0436 - val_acc: 0.4099

Epoch 00003: val_loss did not improve from 1.87818
Epoch 4/25
781/781 [==============================] - 1175s 2s/step - loss: 5.6777 - acc: 0.4649 - val_loss: 1.3601 - val_acc: 0.5665

Epoch 00004: val_loss improved from 1.87818 to 1.36011, saving model to /subset_data/ResNet50/MobileNetV2_201903